<a href="https://colab.research.google.com/github/myllanes/Introduction-to-Deep-Learning/blob/main/HW5_1_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#Michael Yllanes
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt
import math

# Sample text
text = "Next character prediction is a fundamental task in the field of natural language processing (NLP) that involves predicting the next character in a sequence of text based on the characters that precede it. This task is essential for various applications, including text auto-completion, spell checking, and even in the development of sophisticated AI models capable of generating human-like text. At its core, next character prediction relies on statistical models or deep learning algorithms to analyze a given sequence of text and predict which character is most likely to follow. These predictions are based on patterns and relationships learned from large datasets of text during the training phase of the model. One of the most popular approaches to next character prediction involves the use of Recurrent Neural Networks (RNNs), and more specifically, a variant called Long Short-Term Memory (LSTM) networks. RNNs are particularly well-suited for sequential data like text, as they can maintain information in 'memory' about previous characters to inform the prediction of the next character. LSTM networks enhance this capability by being able to remember long-term dependencies, making them even more effective for next character prediction tasks. Training a model for next character prediction involves feeding it large amounts of text data, allowing it to learn the probability of each character's appearance following a sequence of characters. During this training process, the model adjusts its parameters to minimize the difference between its predictions and the actual outcomes, thus improving its predictive accuracy over time. Once trained, the model can be used to predict the next character in a given piece of text by considering the sequence of characters that precede it. This can enhance user experience in text editing software, improve efficiency in coding environments with auto-completion features, and enable more natural interactions with AI-based chatbots and virtual assistants. In summary, next character prediction plays a crucial role in enhancing the capabilities of various NLP applications, making text-based interactions more efficient, accurate, and human-like. Through the use of advanced machine learning models like RNNs and LSTMs, next character prediction continues to evolve, opening new possibilities for the future of text-based technology."

# Creating character vocabulary
chars = sorted(list(set(text)))
ix_to_char = {i: ch for i, ch in enumerate(chars)}
char_to_ix = {ch: i for i, ch in enumerate(chars)}

# Preparing the dataset
max_length = 10  #input sequences
X = []
y = []
for i in range(len(text) - max_length):
    sequence = text[i:i + max_length]
    label = text[i + max_length]
    X.append([char_to_ix[char] for char in sequence])
    y.append(char_to_ix[label])

X = np.array(X)
y = np.array(y)

# Splitting the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Converting data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.long)

# Encoding
class Encoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(Encoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Transformer model
class CharTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, nhead=2): # Input Number layers and heads
        super(CharTransformer, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.pos_encoder = Encoding(hidden_size, max_len=max_length)

        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead, hidden_size)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Embedding and encoding
        x = self.embedding(x)
        x = x.permute(1, 0, 2)
        x = self.pos_encoder(x)

        # Transformer
        output = self.transformer_encoder(x)

        # output for prediction
        output = output[-1, :, :]
        output = self.fc(output)
        return output

# Hyperparameters
hidden_size = 128
learning_rate = 0.005
epochs = 200 # epochs

# Model, loss, and optimizer
model = CharTransformer(len(chars), hidden_size, len(chars))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Calculate model size (number of parameters)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

model_size = count_parameters(model)
print(f"Model size: {model_size} parameters")

# Training the model
start_time = time.time()
train_losses = []
val_losses = []
val_accuracies = []

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())

    # Validation
    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, y_val)
        val_losses.append(val_loss.item())
        _, predicted = torch.max(val_output, 1)
        val_accuracy = (predicted == y_val).float().mean()
        val_accuracies.append(val_accuracy.item())

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}, Validation Accuracy: {val_accuracy.item():.4f}')

# Calculate total training time
training_time = time.time() - start_time
print(f"Training time: {training_time:.2f} seconds")


# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-max_length:]], dtype=torch.long).unsqueeze(0)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

# Predicting the next character
test_str = "Samp"
predicted_char = predict_next_char(model, char_to_ix, ix_to_char, test_str)
print(f"Predicted next character for '{test_str}': '{predicted_char}'")

# report
print(f"Final Training Loss: {train_losses[-1]:.4f}")
print(f"Final Validation Loss: {val_losses[-1]:.4f}")
print(f"Final Validation Accuracy: {val_accuracies[-1]:.4f}")
print(f"Training Time: {training_time:.2f} seconds")
print(f"Model Size: {model_size} parameters")


Model size: 210476 parameters
Epoch 10, Loss: 2.4652, Validation Loss: 2.4615, Validation Accuracy: 0.2920
Epoch 20, Loss: 2.0165, Validation Loss: 2.2517, Validation Accuracy: 0.3592
Epoch 30, Loss: 1.6725, Validation Loss: 2.2114, Validation Accuracy: 0.4118
Epoch 40, Loss: 1.3783, Validation Loss: 2.3117, Validation Accuracy: 0.4139
Epoch 50, Loss: 1.1616, Validation Loss: 2.3939, Validation Accuracy: 0.4265
Epoch 60, Loss: 1.0088, Validation Loss: 2.5333, Validation Accuracy: 0.4223
Epoch 70, Loss: 0.8544, Validation Loss: 2.6951, Validation Accuracy: 0.4517
Epoch 80, Loss: 0.7452, Validation Loss: 2.7693, Validation Accuracy: 0.4391
Epoch 90, Loss: 0.6641, Validation Loss: 2.9735, Validation Accuracy: 0.4454
Epoch 100, Loss: 0.6253, Validation Loss: 3.0600, Validation Accuracy: 0.4349
Epoch 110, Loss: 0.5822, Validation Loss: 3.2203, Validation Accuracy: 0.4412
Epoch 120, Loss: 0.5096, Validation Loss: 3.2646, Validation Accuracy: 0.4433
Epoch 130, Loss: 0.4990, Validation Loss: 3